In [1]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# **Install all required libraries**

1. Huggingface libraries to use Mistral 7B LLM (Open Source LLM)

2. LangChain library to call LLM to generate reposne based on the prompt

In [4]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 5.2 MB/s eta 0:00:00


Import required Libraries and check whether we have access to GPU or not.

We must be getting following output after running the cell

Device: cuda

Tesla T4

In [5]:
#from typing import List
import os
import time
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

import torch

from langchain.llms import HuggingFacePipeline


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4



# **We are using Mistral 7 Billion parameter LLM (Open source from HuggingFace)**

If you're new to Hugging Face or machine learning tasks, the following lines of code introduce some scary concepts. This Mistral 7B LLM, although big in size (approximately 14 GB), presents a challenge when loading on a Colab notebook, requiring larger GPUs and making it incompatible with the free T4 GPU.

To address this issue, we employ a sharded version of the Mistral LLM. This version is loaded in smaller increments (approximately 1.9 GB each) in the notebook and subsequently aggregated into a single file. This allows us to seamlessly utilize the LLM on a free Colab GPU (T4 GPU) without incurring any costs for learning purposes.

These lines of code incorporate several important concepts, which may initially seem complex but are easily comprehensible:

1. BitsAndBytesConfig:

  - load_in_4bit
  - bnb_4bit_use_double_quant
  - bnb_4bit_quant_type
  - bnb_4bit_compute_dtype

2. AutoModelForCausalLM.from_pretrained

3. AutoTokenizer.from_pretrained

Despite the use of some technical terminology, don't be discouraged. In the coming days, we'll delve into these concepts in an approachable manner, breaking down the complexities. This includes a detailed understanding of the mentioned configurations, loading procedures, and tokenization processes. Embrace the learning journey; it's simpler than it seems.








In [6]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# **Creating pipelines to run LLM at Colab notebook**

It uses

1. Huggingface transformers pipeline
  - This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

  - We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.


2. LangChain HuggingFacePipeline
  - Integrating "transformers.pipeline" with LangChain.
  - You may be thinking why we need this. We will be using LangChain extensively later and this is first step to integrate our LLM with LangChain


In [7]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    batch_size=4,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=600,
    temperature = 0.3,
    do_sample=True,
)
text_generation_pipeline.tokenizer.pad_token_id = text_generation_pipeline.model.config.eos_token_id
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## **Text Summarization Task**

In [8]:
#Librarires to count token programmatically
!pip install -q -U tiktoken
!pip install -q -U openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-

# **Option-1**
LangChain StuffDocumentChain

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [10]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.document import Document


def stuff_text_summarization():
  text = """It was a Thursday, but it felt like a Monday to John. And John loved Mondays. He thrived at work. He dismissed the old cliché of dreading Monday mornings and refused to engage in water-cooler complaints about “the grind” and empty conversations that included the familiar parry “How was your weekend?” “Too short!”. Yes, John liked his work and was unashamed.

  I should probably get another latte. I’ve just been sitting here with this empty cup. But then I’ll start to get jittery. I’ll get a decaf. No, that’s stupid, it feels stupid to pay for a decaf. I can’t justify that.

  John was always impatient on the weekends; he missed the formal structure of the business week. When he was younger he used to stay late after school on Fridays and come in early on Mondays, a pattern his mother referred to with equal parts admiration and disdain as “studying overtime.”

  Jesus, I’ve written another loser.

  Now, John spent his weekends doing yard work at the Tudor house Rebecca left him after their divorce. Rebecca, with her almond eyes—both in shape and in color—could never be his enemy.

  That barista keeps looking at me. She’ll probably ask me to leave if I don’t buy something. She’s kind of attractive. Not her hair—her hair seems stringy—but her face is nice. I should really buy something.

  Their divorce was remarkably amicable. In fact, John would often tell his parents, “Rebecca and I are better friends now than when we were married!” In fact, John looked forward to the days when he and Rebecca, with their new partners, would reminisce about their marriage, seeing it in a positive light, like two mature adults.

  Maybe I’ll just get a pumpkin-spice loaf. That way I can still sit here without going through a whole production of buying a coffee and giving my name and feeling like an asshole while it gets made.

  But if John were being honest, the house did get lonely on the weekends. Rebecca’s parents had been generous enough to leave John the house even though they had paid for it. John was still struggling to get his short-story writing—I mean, his painting—career off the ground, and Rebecca and her family had been more than supportive, even during the breakup."""

  token_count = num_tokens_from_string(text, "cl100k_base")
  print(f'Token Count: {token_count}')

  #Converting text to LangChain documents so that StuffDocumentsChain can understand Input
  documents = Document(page_content=text, metadata={"source": "local"})

  # Define prompt with prompt template
  prompt_template = """Write a concise summary of the following:
  "{docs}"
  CONCISE SUMMARY:"""
  prompt = PromptTemplate.from_template(prompt_template)

  # Define LLM chain
  llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

  # Define StuffDocumentsChain
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="docs")

  #Get the Summary of text by invoking StuffDocumentsChain
  summary = stuff_chain.invoke([documents])
  return summary['output_text']

In [11]:
stuff_summary = stuff_text_summarization()
#Print the output text after getting summary
print(stuff_summary)

Token Count: 487
 John enjoys his job and looks forward to Mondays. He misses the structure of the weekdays on weekends and does yard work at his ex-wife's house. He struggles to balance his passion for art with making a career out of it.


# **Document Loaders**
Document loaders in LangChain enable the extraction of data from various sources.

In [12]:
from langchain.document_loaders import PyPDFLoader

def load_pdf():
    # Load the pdf file
    loader = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/ImpactofIndianPremierLeagueonTestCricketinIndia.pdf')
    documents = loader.load()
    token_count = num_tokens_from_string(str(documents), "cl100k_base")
    print(f'PDF Token Count: {token_count}')
    return documents, token_count


# **Option - 2**

LangChain MapReduce (For large text where LLM has context length limitation)

In [13]:
from langchain.chains import MapReduceDocumentsChain, LLMChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

def map_reduce_summarize_document():

    #Load PDF using PyPDF
    documents, token_count = load_pdf()

    # Map Prompt template and LLM Chain
    map_prompt_template = """[INST] The following is a part of an pdf document:
    {docs}
    Based on this, please identify the main points.
    Answer:  [/INST] """
    map_prompt = PromptTemplate.from_template(map_prompt_template)
    map_chain = LLMChain(llm=mistral_llm, prompt=map_prompt)


    # Reduce Prompt template and LLM Chain
    reduce_prompt_template = """[INST] The following is set of summaries from the article:
    {doc_summaries}
    Take these and distill it into a final, consolidated summary of the main points.
    Construct it as a well organized summary of the main points and should be between 3 and 5 paragraphs.
    Answer:  [/INST] """
    reduce_prompt = PromptTemplate.from_template(reduce_prompt_template)
    reduce_chain = LLMChain(llm=mistral_llm, prompt=reduce_prompt)


    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="doc_summaries"
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    #Document Transformers in LangChain are crucial for manipulating documents,
    #performing tasks like splitting, combining, and filtering to adapt to model
    #context or specific application needs. The RecursiveCharacterTextSplitter
    #is a versatile tool allowing customization of parameters such as chunk size,
    #overlap, and starting index for efficient text segmentation.

    # Split documents into chunks using RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(documents)

    # Run the chain
    result = map_reduce_chain.invoke(split_docs, return_only_outputs=True)
    return result['output_text']

In [14]:
summary_response = map_reduce_summarize_document()

PDF Token Count: 7268


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2169 > 1024). Running this sequence through the model will result in indexing errors


In [15]:
print(summary_response)

1. The article discusses the impact of the Indian Premier League (IPL) on Test cricket in India.
2. The authors argue that despite previous research on the topic, there is still a gap in our understanding of the specific changes brought about by IPL in Indian cricket.
3. The paper aims to fill this gap by examining the effects of IPL on Test cricket in India.
4. The significance of the study lies in its potential to provide valuable insights into the current state of Test cricket in India, the popularity of different formats of the game among youth, and the perception of cricket among the population.
5. Additionally, the study seeks to analyze the economic impact of IPL on the nation and understand the unique features that make it different from other leagues.
6. Finally, the paper aims to contribute to the development of measures and training plans for coaches based on the findings of the study.
7. The study used the Chi-square test to determine if there is a significant difference in